### 75.06/95.58 Organización de Datos: Trabajo Práctico 1 ###
#### Primer Cuatrimestre de 2020 ####



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import STOPWORDS
import string
%matplotlib inline

tweets = pd.read_csv('train.csv') 
tweets.tail()

In [ ]:
tweets.info()

In [ ]:
tweets['text'].describe()

In [ ]:
tweets['location'].describe()

In [ ]:
tweets['keyword'].describe()

In [ ]:
duplicados = tweets.duplicated(subset = 'text', keep = False)
duplicados.value_counts()

In [ ]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

Similaridad de textos falsos entre  si y reales entre si

In [ ]:
f = tweets.groupby("target").get_group(0)
v = tweets.groupby("target").get_group(1)

In [ ]:
from datasketch import MinHash,MinHashLSH
import re

def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

def get_minhash(data):
    minhash = []
    for row in data.iterrows():
        text = row[1][3]
        id = row[1][0]
        tokens = preprocess(text)
        m = MinHash(num_perm=512)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append([id,m])
    return minhash

In [ ]:
minhashs_f = get_minhash(f)
minhashs_v = get_minhash(v)

In [ ]:
def calcular_cercania(minhashs):
    final = []
    for i in range(1,100):

        lsh = MinHashLSH(threshold=i/100, num_perm=512)
        for minhash in minhashs:
            lsh.insert(minhash[0],minhash[1])

        values = []
        for minhash in minhashs:
            values.append(len(lsh.query(minhash[1])))
        final.append(values)
    return final

In [ ]:
cerca_segun_treshold_f = calcular_cercania(minhashs_f)
cerca_segun_treshold_v = calcular_cercania(minhashs_v)

In [ ]:
real_avg = []
fake_avg = []
indx = []

for i in range(1,99): 
    real_avg.append(sum(cerca_segun_treshold_v[i])//len(cerca_segun_treshold_v[i]))
    fake_avg.append(sum(cerca_segun_treshold_f[i])//len(cerca_segun_treshold_f[i]))
    indx.append(i)


In [ ]:
real_and_fake = pd.DataFrame(real_avg,index = indx)
real_and_fake.columns = ["AVG Match Real"]
real_and_fake["AVG Match Fake"] = fake_avg
real_and_fake.reset_index(inplace=True)
real_and_fake = real_and_fake.head(40)

In [ ]:
ax = real_and_fake.plot.scatter(x="AVG Match Real",y="index",color="r",label="Reales",figsize=(12,10))
real_and_fake.plot.scatter(x="AVG Match Fake",y="index",color="b",label="Falsos",ax=ax)
real_and_fake.plot(x="AVG Match Fake",y="index",color="b",label="",ax=ax)
real_and_fake.plot(x="AVG Match Real",y="index",color="r",label="",ax=ax)
plt.legend(loc='best')

ax.set_xlabel("Promedio de tweets similares")
ax.set_ylabel("Minimo porcentaje de similitud")
ax.plot()

TOP 50 DESASTRES COMENTADOS EN LOS TWEETS

In [ ]:
desastres = tweets.groupby("keyword").agg({"target":["count","mean"]})
level0 = desastres.columns.get_level_values(0)
level1 = desastres.columns.get_level_values(1)
desastres.columns = level0 + "_" + level1
desastres.sort_values(by="target_count",ascending = False,inplace = True)
desastres = desastres.head(50) #TOP 50

In [ ]:
fig = plt.figure(figsize=(20,20))
grafico = sns.barplot(data = desastres,x = "target_count",y = desastres.index)
grafico.set_title("Top 50 desastres comentados en tweets",fontsize = 14)
grafico.set_xlabel("Cantidad de veces mencionado",fontsize = 14)
grafico.set_ylabel("Desastre",fontsize = 14)
grafico.plot()

TOP PALABRAS EN LOS TWEETS

In [ ]:
def by_word_count(dataframe):
    word_list = dataframe["text"].str.split()
    all_stopwords_gensim = STOPWORDS.union(set(string.punctuation))
    filtered = [word.lower() for word in np.concatenate(word_list.values) if not word in all_stopwords_gensim]
    words = pd.DataFrame(filtered,columns = ["word"])
    words["amount"] = 1
    by_count = words.groupby(["word"]).agg({"amount":["count"]})
    level0 = by_count.columns.get_level_values(0)
    level1 = by_count.columns.get_level_values(1)
    by_count.columns = level0 + "_" + level1
    by_count.sort_values(by="amount_count",ascending = False,inplace = True)
    return by_count.head(50)

In [ ]:
true = tweets["target"] == 1
true = by_word_count(tweets[true])

false = tweets["target"] == 0
false = by_word_count(tweets[false])

In [ ]:
fig = plt.figure(figsize=(20,20))
grafico = sns.barplot(data = true,x = "amount_count",y=true.index)
grafico.set_title("Top 10 palabras usadas en tweets verdaderos",fontsize = 14)
grafico.set_xlabel("Cantidad de veces",fontsize = 14)
grafico.set_ylabel("Palabra",fontsize = 14)
grafico.plot()

In [ ]:
fig = plt.figure(figsize=(20,20))
grafico = sns.barplot(data = false,x = "amount_count",y=false.index)
grafico.set_title("Top 10 palabras usadas en tweets falsos",fontsize = 14)
grafico.set_xlabel("Cantidad de veces",fontsize = 14)
grafico.set_ylabel("Palabra",fontsize = 14)
grafico.plot()

Hashtags más usados en tweets reales y falsos

In [ ]:
tweets_with_ht = tweets.loc[tweets['text'].str.contains('#')]
tweets_with_ht['hashtags'] = tweets_with_ht['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
tweets_hashtags = tweets_with_ht.explode('hashtags')
tweets_hashtags.dropna(subset = ["hashtags"],inplace = True)
tweets_hashtags["hashtags"] = tweets_hashtags["hashtags"].str.lower()

In [ ]:
true_by_hashtags = tweets_hashtags.groupby("target").get_group(1)
false_by_hashtags = tweets_hashtags.groupby("target").get_group(0)

In [ ]:
true_hashtags = ' '.join(true_by_hashtags["hashtags"].str.lower())
fake_hashtags = ' '.join(false_by_hashtags["hashtags"].str.lower())

In [ ]:
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import imageio

twitter_coloring = imageio.imread("img/twitter.png",pilmode='RGB')
wc = WordCloud(width = 1920,height = 1080,background_color = "black",mask=twitter_coloring)
image_colors = ImageColorGenerator(twitter_coloring)

In [ ]:
wc.generate(fake_hashtags)
fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(wc, interpolation="bilinear")
ax.set_axis_off()
plt.tight_layout(pad=0)
plt.show()

In [ ]:
wc.generate(true_hashtags)
fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(wc, interpolation="bilinear")
ax.set_axis_off()
plt.tight_layout(pad=0)
plt.show()

In [ ]:
top_real_hashtags = true_by_hashtags.groupby('hashtags').agg({'target':'count'}).nlargest(50, columns='target')
top_real_hashtags.columns = ["real"]
top_fake_hashtags = false_by_hashtags.groupby('hashtags').agg({'target':'count'}).nlargest(50, columns="target")
top_fake_hashtags.columns = ["fake"]

In [ ]:
all_hashtags = pd.concat([top_real_hashtags,top_fake_hashtags],join="outer",axis=1)
all_hashtags.fillna(0,inplace=True)
all_hashtags['total'] = all_hashtags['real'] + all_hashtags['fake']
top_20_hashtags = all_hashtags.nlargest(20, "total")
ax = top_20_hashtags.sort_values(by='total').loc[:, ['real', 'fake']].plot(kind='barh', figsize=(10, 10))